In [ ]:
# https://towardsdatascience.com/multi-class-text-classification-with-doc2vec-logistic-regression-9da9947b43f4

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from sklearn import utils
from sklearn.model_selection import train_test_split
import gensim
from sklearn.linear_model import LogisticRegression
from gensim.models.doc2vec import TaggedDocument
import re
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline
from pathlib import Path
import pandas as pd
import numpy as np

In [ ]:
import os
from os import listdir

In [ ]:
DATA = "/kaggle/input"

In [ ]:
# explore the input data
# for dirname, _, filenames in os.walk(DATA):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

In [ ]:
train_directory = os.path.join(DATA,'feedback-prize-2021','train')

In [ ]:
test_directory = os.path.join(DATA,'feedback-prize-2021','test')

In [ ]:
glove_directory = os.path.join(DATA,'glovedata')

In [ ]:
root_directory = os.path.join(DATA,'feedback-prize-2021')

In [ ]:
# explore the word2vec datasets
for dirname, _, filenames in os.walk(glove_directory):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
train_filename = "train.csv"

In [ ]:
train_filepath = os.path.join(root_directory,train_filename)

In [ ]:
essay_files = listdir(train_directory)

In [ ]:
len(essay_files)

In [ ]:
# to bring in the training set spreadsheet (.csv)

In [ ]:
train_df = pd.read_csv(train_filepath)

In [ ]:
train_df.head()

In [ ]:
print(len(train_df))

In [ ]:
def retrieveWordage(essay_id,directory):
  essay_filename = essay_id + ".txt"
  essay_filepath = os.path.join(directory,essay_filename)
  with open(essay_filepath) as file:
    file_contents = file.read()
  #try:
  #  essay_text = pd.read_table(essay_filepath,header=0,engine='python',error_bad_lines=False)
  #except:
  #  return None
  #essay_text.columns = ['narrative']
  #return essay_text
  essay_text = []
  essay_line = ""
  for c in file_contents:
    if c == '\n':
      if len(essay_line) > 1:
        essay_text.append(essay_line)
      essay_line = ""
    else:
      essay_line += c
  essay_text.append(essay_line)
  essay_pd = pd.DataFrame(essay_text)
  essay_pd.columns = ['narrative']
  return essay_pd

In [ ]:
wordage_test = retrieveWordage("0027FC00C35B",train_directory)
print(wordage_test)

In [ ]:
def cleanText2(item):
  item2 = ""
  for ch in item:
      if ch == ' ' or (ch >= 'a' and ch <= 'z') or (ch >= 'A' and ch <= 'Z') or (ch >= '0' and ch <= '9'):
          if ch >= 'A' and ch <= 'Z':
              item2 += ch.lower()
          else:
              item2 += ch
  return item2

In [ ]:
def retrieveEssay(essay_id,directory):
  essay_text = retrieveWordage(essay_id,directory)
  if essay_text is None:
        return None
  essay_count = len(essay_text)
  essay = ""
  for i in range(0,essay_count-1):
      essay += cleanText2(essay_text.iloc[i,0])
  return essay

In [ ]:
def retrieveRawEssay(essay_id,directory):
  essay_text = retrieveWordage(essay_id,directory)
  if essay_text is None:
      return None
  essay_count = len(essay_text)
  essay = ""
  for i in range(0,essay_count-1):
      item = essay_text.iloc[i,0]
      essay += item
  return essay

In [ ]:
essay_test = retrieveEssay("423A1CA112E2",train_directory)
print(essay_test)

In [ ]:
from tensorflow.data import Dataset

In [ ]:
from tensorflow.data import TextLineDataset

In [ ]:
def retrieveDataset(essay_id,directory):
  essay_array = []
  essay_text = retrieveWordage(essay_id,directory)
  if essay_text is None:
    return None
  essay_count = len(essay_text)
  #essay = ""
  for i in range(0,essay_count-1):
      essay_array.append(cleanText2(essay_text.iloc[i,0]))
  #essay = essay_text.iloc[0,0]
  dataset = Dataset.from_tensor_slices(essay_array)
  return dataset

In [ ]:
test_dataset = retrieveDataset("423A1CA112E2",train_directory)
print(test_dataset)
for element in test_dataset:
    print(element)

In [ ]:
# https://www.tensorflow.org/api_docs/python/tf/keras/layers/TextVectorization

In [ ]:
from tensorflow.keras.layers import TextVectorization

In [ ]:
# Create the layer.
vectorize_layer = TextVectorization(
 max_tokens=5000,
 output_mode='int',
 output_sequence_length=500)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input
from tensorflow import string as tf_string

In [ ]:
def retrieveDiscourse(essay_id,discourse_id,discourse_start,discourse_end,directory):
  essay_text = retrieveRawEssay(essay_id,directory)
  essay_count = len(essay_text)
  i = 0
  discourse_text = ""
  for ch in essay_text:
     #print(ch) 
     if i >= discourse_start - 2 and i <= discourse_end + 1:
        discourse_text += ch
     i += 1
  return cleanText2(discourse_text)

In [ ]:
test_discourse = retrieveDiscourse("423A1CA112E2",0,8,229,train_directory)
print(test_discourse)
print(len(test_discourse))

In [ ]:
# https://colab.research.google.com/github/keras-team/keras-io/blob/master/examples/nlp/ipynb/pretrained_word_embeddings.ipynb#scrollTo=Q6j-LRMFc-AR

In [ ]:
#!wget http://nlp.stanford.edu/data/glove.6B.zip
#!unzip -q glove.6B.zip

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.initializers import Constant
from tensorflow.keras import layers
from tensorflow.keras import Input
from tensorflow.keras import Model

In [ ]:
essays_group = train_df.groupby(['id'])

In [ ]:
class_dict = {}
samples = []
labels = []
class_index = 0
for essay_id, essay_pd in essays_group:  
  essay = retrieveEssay(essay_id,train_directory)
  if essay is None:
    break
  discourse_count = len(essay_pd)
  for i in range(0,discourse_count):
    item = essay_pd.iloc[i,]
    discourse_id = item['discourse_id']
    discourse_start = item['discourse_start']
    discourse_end = item['discourse_end']
    discourse_type = item['discourse_type']
    discourse_type = discourse_type.lower()
    if class_dict.get(discourse_type) == None:
      class_name_index = class_index
      class_dict[discourse_type] = class_name_index
      class_index += 1
    else:
      class_name_index = class_dict.get(discourse_type)
    discourse = retrieveDiscourse(essay_id,discourse_id,discourse_start,discourse_end,train_directory)
    samples.append(discourse)
    labels.append(class_name_index)

In [ ]:
essay_array = []
samples_len = len(samples)
for i in range(0,samples_len):
  essay_text = samples[i]
  if essay_text != "":
      essay_array.append(essay_text)
vocab = Dataset.from_tensor_slices(essay_array)

In [ ]:
# Now that the vocab layer has been created, call `adapt` on the text-only
# dataset to create the vocabulary. You don't have to batch, but for large
# datasets this means we're not keeping spare copies of the dataset.
vectorize_layer.adapt(vocab)

In [ ]:
voc = vectorize_layer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [ ]:
print(len(word_index))

In [ ]:
num_tokens = len(voc) + 2
embedding_dim = 100
hits = 0
misses = 0

In [ ]:
path_to_glove_file = os.path.join(glove_directory,"glove.6B.100d.txt")

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

In [ ]:
# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

In [ ]:
embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=Constant(embedding_matrix),
    trainable=False,
)

In [ ]:
class_names = []
for key in class_dict:
  print(key)
  class_names.append(key)

In [ ]:
print(class_dict)
print(class_names)

In [ ]:
# Extract a training & validation split
validation_split = 0.2
num_validation_samples = int(validation_split * len(samples))
train_samples = samples[:-num_validation_samples]
test_samples = samples[-num_validation_samples:]

In [ ]:
print(len(train_samples))
print(len(test_samples))

In [ ]:
train_labels = labels[0:len(train_samples)]
test_labels = labels[len(train_samples):(len(train_samples)+len(test_samples))]

In [ ]:
# The class names should be the categories we are looking for.

In [ ]:
int_sequences_input = Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.Conv1D(128, 5, activation="relu")(embedded_sequences)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)
preds = layers.Dense(len(class_names), activation="softmax")(x)
model = Model(int_sequences_input, preds)
model.summary()

In [ ]:
x_train = vectorize_layer(np.array([[s] for s in train_samples])).numpy()
x_test = vectorize_layer(np.array([[s] for s in test_samples])).numpy()

In [ ]:
y_train = np.array([[s] for s in train_labels])
y_test = np.array([[s] for s in test_labels])

In [ ]:
model.compile(
    loss="sparse_categorical_crossentropy", optimizer="rmsprop", metrics=["acc"]
)

In [ ]:
#model.fit(x_train, y_train, batch_size=128, epochs=20, validation_data=(x_test, y_test))
model.fit(x_train, y_train, batch_size=128, epochs=20)
#model.fit(x_train,y_train)

In [ ]:
# Kaggle has a read-only file system so we might have to find another way to persist!
# path_to_model = os.path.join(DATA,'feedback-prize-2021','model')
# model.save(path_to_model)

In [ ]:
y_result = model.predict(x_test)

In [ ]:
print(y_result)

In [ ]:
yt0 = y_test[1]

In [ ]:
y_rounded = np.round(y_result,1)

In [ ]:
y0 = y_rounded[1]

In [ ]:
print(y0)

In [ ]:
print(yt0)

In [ ]:
print(len(y_rounded))
print(len(y_test))

In [ ]:
for i in range(0,10):
    y_out = y_rounded[i]
    yt = y_test[i]
    print(y_out)
    print(yt)

In [ ]:
# Create Statistics about Where Discourses are in Essays
# To do this we first create a pandas dataframe 'essay_statistics', per essay it lists
# essay_id,essay_length (in characters),discourse_count
class_dict = {}
samples = []
labels = []
class_index = 0
length_of_essay = 0
stats = []
for essay_id, essay_pd in essays_group: 
  stats_entry = []
  stats_entry.append(essay_id)
  essay = retrieveEssay(essay_id,train_directory)
  if essay is None:
    break
  length_of_essay = len(essay)
  stats_entry.append(length_of_essay)
  discourse_count = len(essay_pd)
  stats_entry.append(discourse_count)
  stats.append(stats_entry)
essay_statistics = pd.DataFrame(stats)

In [ ]:
essay_statistics.head()

In [ ]:
essay_statistics.columns = ['essay_id','essay_length','no_of_discourses']

In [ ]:
essay_statistics.head()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
x = essay_statistics['essay_length']
y = essay_statistics['no_of_discourses']

In [ ]:
plt.plot(x,y)

In [ ]:
# https://realpython.com/numpy-scipy-pandas-correlation-python/#visualization-of-correlation

In [ ]:
x.corr(y)

In [ ]:
y.corr(x)

In [ ]:
from scipy import stats as sci_stats

In [ ]:
#https://realpython.com/numpy-scipy-pandas-correlation-python/#visualization-of-correlation
slope, intercept, r, p, stderr = sci_stats.linregress(x, y)

In [ ]:
print(slope)
print(intercept)
print(r)
print(p)
print(stderr)

In [ ]:
def returnExpectedDiscourses(essay_length):
    yyy = intercept + essay_length * slope
    return yyy

In [ ]:
discourse_num_err = np.round(r,0)

In [ ]:
print(discourse_num_err)

In [ ]:
print(returnExpectedDiscourses(1000))

In [ ]:
sample_x = []
sample_y = []
for i in range(0,8000,500):
    yy = returnExpectedDiscourses(i)
    sample_x.append(i)
    sample_y.append(yy)

In [ ]:
plt.plot(sample_x,sample_y)

In [ ]:
essay_test = retrieveEssay("423A1CA112E2",train_directory)
print(essay_test)

In [ ]:
length_of_essay = len(essay_test)
expected_discourses = returnExpectedDiscourses(length_of_essay)
print(length_of_essay)
print(expected_discourses)

In [ ]:
expected_discourses = 13

In [ ]:
# Start out with all discourses of even length
test_discourse = []
length_of_discourses = round(length_of_essay/expected_discourses,0)
print(length_of_discourses)

In [ ]:
text_start = 0
discourse_element = []
test_discourse = []
for i in range(1,int(expected_discourses)):
    #print(i)
    text_end = text_start + length_of_discourses
    discourse_element.append(text_start)
    discourse_element.append(text_end)
    text_start = text_end + 1
    test_discourse.append(discourse_element)
    discourse_element = []
text_start = text_end + 1
text_end = length_of_essay
discourse_element.append(text_start)
discourse_element.append(text_end)
test_discourse.append(discourse_element)

In [ ]:
# Given an expected length in characters
# the essay
# the offset
# returns a Prediction String as a numerical list
# Of word offsets into the essay
def returnPredictionString()

In [ ]:
# We wish to have a good way to decide if a given 'fit' is good enough
# Given the results of model.predict of a given discourse using the prediction string above.
# This is a numerical value (statistic) between 0 and 1: Power
# Not arbitrary!
def returnScore()

In [ ]:
# Given a prediction string and a score, how do we know if the string can be improved.
# Do we remove a word and add another?
# The final test has to adjust the discourses appropriately to create the correct answer.
# Should we use the example prediction strings to somehow infer whether we are selecting the correct words?
# Haven't we already using the discourses to build the model?
#
# Certainly we could feed the proper discourses back into our model using the prediction string
# and see if we achieve the highest score that way
#
# Using the training set, can we create a set of adjustments that increase the score by adjusting the 
# predition string from the equal set where we have chopped the essay into the appropriate number of discourses
# to the prediction strings indicated by the high score.

In [ ]:
def returnDiscouseFromPredictionString():

In [ ]:
print(int(expected_discourses))
print(length_of_essay)
print(test_discourse)

In [ ]:
print(len(essay_test))

In [ ]:
x_discourses = []
for discourse in test_discourse:
    print(discourse)
    start_text = int(discourse[0])
    end_text = int(discourse[1])
    discourse_element = ""
    for i in range(start_text,end_text):
        discourse_element += essay_test[i]
    x_discourses.append(discourse_element)

In [ ]:
print(x_discourses)

In [ ]:
x_test = vectorize_layer(np.array([[s] for s in x_discourses])).numpy()

In [ ]:
print(x_test)

In [ ]:
y_result = model.predict(x_test)

In [ ]:
print(y_result)

In [ ]:
# Given the test result, compute the score
def computeScore(y_result):
    cumulative_score = 0
    number_of_items = 0
    for item in y_result:
        max_score = 0
        for score in item:
            if max_score < score:
                max_score = score
        cumulative_score += max_score
        number_of_items += 1
    #print(number_of_items)
    return cumulative_score / number_of_items

computeScore(y_result)

In [ ]:
# 8 
# 0.5768586434423923
# 9
# 0.5594
# 10
# 0.4849
# 11
# 0.4849
# 12
# 0.6639
# 13
# 0.698

In [ ]:
es = train_df[train_df['id']=="423A1CA112E2"]
print(es)

In [ ]:
print(len(y_result))

In [ ]:
# We are ready to run the test!

In [ ]:
# explore the word2vec datasets
for dirname, _, filenames in os.walk(test_directory):
    for filename in filenames:
        essay_id = (filename.split('.'))[0]
        print(essay_id)
        essay_test = retrieveEssay(essay_id,test_directory)
        # print(essay_test)
        length_of_essay = len(essay_test)
        expected_discourses = returnExpectedDiscourses(length_of_essay)
        print(length_of_essay)
        print(expected_discourses)
        length_of_discourses = round(length_of_essay/expected_discourses,0)
        print(length_of_discourses)
        text_start = 0
        discourse_element = []
        test_discourse = []
        for i in range(1,int(expected_discourses)):
            #print(i)
            text_end = text_start + length_of_discourses
            discourse_element.append(text_start)
            discourse_element.append(text_end)
            text_start = text_end + 1
            test_discourse.append(discourse_element)
            discourse_element = []
            text_start = text_end + 1
            text_end = length_of_essay
            discourse_element.append(text_start)
            discourse_element.append(text_end)
            test_discourse.append(discourse_element)
        print(int(expected_discourses))
        print(length_of_essay)
        print(test_discourse)  
        x_discourses = []
        for discourse in test_discourse:
            print(discourse)
            start_text = int(discourse[0])
            end_text = int(discourse[1])
            discourse_element = ""
            for i in range(start_text,end_text):
                discourse_element += essay_test[i]
                x_discourses.append(discourse_element)
        print(x_discourses)
        x_test = vectorize_layer(np.array([[s] for s in x_discourses])).numpy()
        print(x_test)
        y_result = model.predict(x_test)
        print(y_result)
        y_score = computeScore(y_result)
        print(y_score)